# SymReg CI2024 Project

In [ ]:
import numpy as np
import random
from tqdm import tqdm
import matplotlib.pyplot as plt

In [ ]:
operators = ['+', '-', '*', '/']
functions = ['np.sin', 'np.cos', 'np.exp', 'np.log', 'np.sqrt', 'np.abs']

## Graph structure definition

In [ ]:
class Node:
    def __init__(self, value, left=None, right=None):
        self.value = value
        self.left = left
        self.right = right

    def __str__(self):
        if self.left is None and self.right is None:
            return str(self.value)
        if self.left is None: # unary operator ( like sin, cos )
            return f"{self.value}({self.right})"
        left_str = str(self.left)
        right_str = str(self.right)
        return f"({left_str} {self.value} {right_str})" # binary operator ( like sum )
    
    def is_operator(self):
        return self.value in operators
    
    def is_function(self):
        return self.value in functions
    
    def is_leaf(self):
        return not self.is_operator() and not self.is_function()
    
    def copy(self):
        return Node(self.value, self.left.copy() if self.left else None, self.right.copy() if self.right else None)
    
    def depth(self):
        # if node is a leaf, depth is 0; elsem depth is 1 + max depth of left/right children
        if self.is_leaf():
            return 0
        left_depth = self.left.depth() if self.left else 0
        right_depth = self.right.depth() if self.right else 0
        return 1 + max(left_depth, right_depth)
    
    def evaluate(self, variable_values):
        # operators
        if self.is_operator():
            left_value = self.left.evaluate(variable_values) if self.left else None
            right_value = self.right.evaluate(variable_values) if self.right else None
            if self.value == '+':
                return left_value + right_value
            elif self.value == '-':
                return left_value - right_value
            elif self.value == '*':
                return left_value * right_value
            elif self.value == '/':
                return left_value / right_value if right_value != 0 else np.inf
        # functions
        elif self.is_function():
            value = self.right.evaluate(variable_values) if self.right else None
            if self.value == 'np.sin':
                return np.sin(value)
            elif self.value == 'np.cos':
                return np.cos(value)
            elif self.value == 'np.exp':
                return np.exp(value)
            elif self.value == 'np.log':
                return np.log(value) if value > 0 else -np.inf
            elif self.value == 'np.sqrt':
                return np.sqrt(value) if value >= 0 else -np.inf
            elif self.value == 'np.abs':
                return np.abs(value)
            
        elif self.value.startswith('x'):
            # variable node like x[1], x[2], etc.
            index = int(self.value[2:-1])  # extract the index from 'x[1]', 'x[2]', etc.
            return variable_values[f"x{index}"]
        
        else:
            # costant node
            try:
                return float(self.value)
            except ValueError:
                raise ValueError(f"Invalid value: {self.value}")

## Genetic Algorithm

### Genetic Algorithm functions

In [ ]:
def generate_formula(depth, n_var):
    
    if depth == 0:
        # return a leaf node, which can be a variable or a constant ( 50% chance for each )
        if random.random() < 0.5:
            # return a variable node like x[1], x[2], etc.
            index = random.randint(1, n_var)
            return Node(f"x{index}")
        else:
            # return a constant node with a random value between -10 and 10 rounded to 3 decimal
            return Node(str(round(random.uniform(-10, 10), 3)))
        
    # choose an operator ( binary ) or function ( unary ) randomly ( again, 50% chance for each )
    if random.random() < 0.5:
        operator = random.choice(operators)
        left = generate_formula(depth - 1, n_var)
        right = generate_formula(depth - 1, n_var)
        return Node(operator, left, right)
    else:
        func = random.choice(functions)
        right = generate_formula(depth - 1, n_var)
        return Node(func, right = right)  # function nodes have only one child
    
def generate_population(x, population_size=250):
    n_var = x.shape[0]
    population = []
    for _ in range(population_size):
        depth = random.randint(n_var, 2 * n_var)
        formula = generate_formula(depth, n_var)
        population.append(formula)
    return population